In [1]:
import time
from rich.pretty import pprint
import logging

logging.getLogger("httpx").setLevel(logging.WARNING)

In [2]:
BASE_URL = "http://localhost:8321"

def create_http_client():
    from llama_stack_client import LlamaStackClient
    return LlamaStackClient(base_url=BASE_URL)

client = create_http_client()

In [3]:
print("Available Models:")
for model in client.models.list():
    pprint(model)


Available Models:


Model(
│   identifier='vllm/qwen2',
│   metadata={},
│   api_model_type='llm',
│   provider_id='vllm',
│   type='model',
│   provider_resource_id='qwen2',
│   model_type='llm'
)

In [4]:
print("Available Shields:")
for shield in client.shields.list():
    pprint(shield)


Available Shields:


Shield(
│   identifier='Prompt-Guard-86M',
│   provider_id='prompt-guard',
│   type='shield',
│   params={},
│   provider_resource_id='Prompt-Guard-86M'
)

### scan raw LLM

In [5]:
benchmark_id = "trustyai_garak::prompt_injection"
client.benchmarks.register(
    benchmark_id=benchmark_id,
    dataset_id=benchmark_id,
    scoring_functions=["string"],
    provider_benchmark_id="prompt_injection",
    provider_id="trustyai_garak_inline",
    metadata={
        "probes": ["promptinject.HijackHateHumans"],
        "timeout": 60*10,
    }
)

In [6]:
job = client.alpha.eval.run_eval(
    benchmark_id=benchmark_id,
    benchmark_config={
        "eval_candidate": {
            "type": "model",
            "model": "vllm/qwen2",
            "sampling_params": {},
        }
     },
)

print(f"Starting job '{job}'")

Starting job 'Job(job_id='garak-job-a4d19a2c-b3e0-4923-b7bb-11dc9f742fb3', status='scheduled', metadata={'created_at': '2025-12-02T00:05:53.150645'})'


In [7]:
def get_job_status(job_id, benchmark_id):
    return client.alpha.eval.jobs.status(job_id=job_id, benchmark_id=benchmark_id)

while True:
    job = get_job_status(job_id=job.job_id, benchmark_id=benchmark_id)
    print(job)

    if job.status in ['failed', 'completed', 'cancelled']:
        print("="*100)
        print(f"Job ended with status: {job.status}")
        break

    time.sleep(20)

Job(job_id='garak-job-a4d19a2c-b3e0-4923-b7bb-11dc9f742fb3', status='in_progress', metadata={'created_at': '2025-12-02T00:05:53.150645', 'started_at': '2025-12-02T00:05:53.153265', 'process_id': '46995', 'running_jobs': '1', 'max_concurrent_jobs': '5'})
Job(job_id='garak-job-a4d19a2c-b3e0-4923-b7bb-11dc9f742fb3', status='in_progress', metadata={'created_at': '2025-12-02T00:05:53.150645', 'started_at': '2025-12-02T00:05:53.153265', 'process_id': '46995', 'running_jobs': '1', 'max_concurrent_jobs': '5'})
Job(job_id='garak-job-a4d19a2c-b3e0-4923-b7bb-11dc9f742fb3', status='in_progress', metadata={'created_at': '2025-12-02T00:05:53.150645', 'started_at': '2025-12-02T00:05:53.153265', 'process_id': '46995', 'running_jobs': '1', 'max_concurrent_jobs': '5'})
Job(job_id='garak-job-a4d19a2c-b3e0-4923-b7bb-11dc9f742fb3', status='in_progress', metadata={'created_at': '2025-12-02T00:05:53.150645', 'started_at': '2025-12-02T00:05:53.153265', 'process_id': '46995', 'running_jobs': '1', 'max_concurre

In [8]:
scores = client.alpha.eval.jobs.retrieve(job_id=job.job_id, benchmark_id=benchmark_id).scores

In [9]:
## extract aggregated scores
aggregated_scores = {k: v.aggregated_results for k, v in scores.items()}
pprint(aggregated_scores)

{
│   'promptinject.HijackHateHumans': {
│   │   'total_attempts': 256.0,
│   │   'benign_responses': 37.0,
│   │   'vulnerable_responses': 219.0,
│   │   'attack_success_rate': 85.55,
│   │   'metadata': {
│   │   │   'avid_taxonomy': {
│   │   │   │   'risk_domain': ['Security'],
│   │   │   │   'sep_view': ['S0403: Adversarial Example'],
│   │   │   │   'lifecycle_view': ['L05: Evaluation']
│   │   │   },
│   │   │   'model': {'type': 'openai.OpenAICompatible', 'name': 'vllm/qwen2'}
│   │   },
│   │   'detector_scores': {'promptinject.AttackRogueString_mean': 85.55}
│   }
}

### scan LLM with appropriate shield

In [10]:
benchmark_id = "trustyai_garak::prompt_injection_with_input_shield"
client.benchmarks.register(
    benchmark_id=benchmark_id,
    dataset_id=benchmark_id,
    scoring_functions=["string"],
    provider_benchmark_id="prompt_injection_with_input_shield",
    provider_id="trustyai_garak_inline",
    metadata={
        "probes": ["promptinject.HijackHateHumans"],
        "timeout": 60*10,
        "shield_ids": ["Prompt-Guard-86M"]
    }
)

All shields in `shield_ids` are considered as input shields.

If you want to use multiple shields as input and output guardrails, you can pass `shield_config` instead of `shield_ids` in the metadata - 

```json
metadata={
        "probes": ["promptinject.HijackHateHumans"],
        "timeout": 600,
        "shield_config": {
            "input": ["Prompt-Guard-86M"], // input shield_ids
            "output": ["Llama-Guard-3-8B"] // output shield_ids
    }

}
```

If you pass both `shield_config` and `shield_ids`, only `shield_ids` is considered.

In [11]:
job = client.alpha.eval.run_eval(
    benchmark_id=benchmark_id,
    benchmark_config={
        "eval_candidate": {
            "type": "model",
            "model": "vllm/qwen2",
            "sampling_params": {
                "max_tokens": 100
            },
        }
     },
)

print(f"Starting job '{job}'")

Starting job 'Job(job_id='garak-job-d7f70611-c9d6-4329-a273-987e262f5d66', status='scheduled', metadata={'created_at': '2025-12-02T00:08:33.529417'})'


In [12]:
def get_job_status(job_id, benchmark_id):
    return client.alpha.eval.jobs.status(job_id=job_id, benchmark_id=benchmark_id)

while True:
    job = get_job_status(job_id=job.job_id, benchmark_id=benchmark_id)
    print(job)

    if job.status in ['failed', 'completed', 'cancelled']:
        print("="*100)
        print(f"Job ended with status: {job.status}")
        break

    time.sleep(20)

Job(job_id='garak-job-d7f70611-c9d6-4329-a273-987e262f5d66', status='in_progress', metadata={'created_at': '2025-12-02T00:08:33.529417', 'started_at': '2025-12-02T00:08:33.530995', 'running_jobs': '1', 'max_concurrent_jobs': '5'})
Job(job_id='garak-job-d7f70611-c9d6-4329-a273-987e262f5d66', status='completed', metadata={'created_at': '2025-12-02T00:08:33.529417', 'started_at': '2025-12-02T00:08:33.530995', 'process_id': '48246', 'scan.report.jsonl': 'file-8f458f2ae70f4c9ebd1f8d79c74258a1', 'scan.avid.jsonl': 'file-e7ea50ee223e47e0a4d6ad49e9cec838', 'scan.log': 'file-80c45d97079647dea8f5bd87a96f8c01', 'scan.report.html': 'file-de9897c374324b5dae26ccaf595c2892', 'scan_result.json': 'file-37bf7cd42f1a4117a320ac1c5ba18ab2', 'completed_at': '2025-12-02T00:08:50.295573', 'running_jobs': '0', 'max_concurrent_jobs': '5'})
Job ended with status: completed


See the prompt_injection score will get drastically reduced because of our input shield

In [13]:
scores = client.alpha.eval.jobs.retrieve(job_id=job.job_id, benchmark_id=benchmark_id).scores

In [14]:
## extract aggregated scores
aggregated_scores = {k: v.aggregated_results for k, v in scores.items()}
pprint(aggregated_scores)

{
│   'promptinject.HijackHateHumans': {
│   │   'total_attempts': 256.0,
│   │   'benign_responses': 256.0,
│   │   'vulnerable_responses': 0.0,
│   │   'attack_success_rate': 0.0,
│   │   'metadata': {
│   │   │   'avid_taxonomy': {
│   │   │   │   'risk_domain': ['Security'],
│   │   │   │   'sep_view': ['S0403: Adversarial Example'],
│   │   │   │   'lifecycle_view': ['L05: Evaluation']
│   │   │   },
│   │   │   'model': {
│   │   │   │   'type': 'function.Single',
│   │   │   │   'name': 'llama_stack_provider_trustyai_garak.shield_scan#simple_shield_orchestrator'
│   │   │   }
│   │   },
│   │   'detector_scores': {'promptinject.AttackRogueString_mean': 0.0}
│   }
}